# Requirements
Windows or Linux<br>
CMake >= 3.12: https://cmake.org/download/<br>
CUDA >= 10.0: https://developer.nvidia.com/cuda-toolkit-archive (on Linux do Post-installation Actions)<br>
OpenCV >= 2.4: use your preferred package manager (brew, apt), build from source using vcpkg or download from OpenCV official site (on Windows set system variable OpenCV_DIR = C:\opencv\build - where are the include and x64 folders image)
cuDNN >= 7.0 https://developer.nvidia.com/rdp/cudnn-archive (on Linux copy cudnn.h,libcudnn.so... as desribed here 
https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html#installlinux-tar , on Windows copy cudnn.h,cudnn64_7.dll, cudnn64_7.lib as desribed here https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html#installwindows )<br>
GPU with CC >= 3.0: https://en.wikipedia.org/wiki/CUDA#GPUs_supported<br>
on Linux GCC or Clang, on Windows MSVC 2017/2019 https://visualstudio.microsoft.com/thank-you-downloading-visual-studio/?sku=Community

# Step 1: Cloning and Building Darknet
The following cells will clone darknet from AlexeyAB's famous repository, adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

Do not worry about any warnings when you run the '!make' cell!

In [2]:
# verify CUDA
# !nvcc --version
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:04_Central_Daylight_Time_2018
Cuda compilation tools, release 10.0, V10.0.130


In [ ]:
# clone Yolov4 darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# Step 2: Download pre-trained YOLOv4 weights
YOLOv4 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv4 on these pretrained classes and get detections.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Step 3: Download COCO Dataset of special categories
categories are person, car, bus, truck

In [ ]:
# import libraries
import io
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import cv2

In [1]:
# download coco dataset annotation zip file
%cd data
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    

--2020-12-04 12:15:38--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... failed: Name or service not known.
wget: unable to resolve host address â€˜images.cocodataset.orgâ€™


In [ ]:
# unzip annotation zip file
!unzip annotations_trainval2017.zip

In [ ]:
!mkdir coco
!mkdir coco/obj
!dir

In [ ]:
dataset_path = 'coco/obj'
coco_path = 'coco'

In [ ]:
# Convert COCO Bounding Box to Yolov4 Format
# COCO Bounding Box format : x(left), y(top), width, height of object
# [477.88, 87.8, 13.7, 33.4]
# Yolov4 Format: central x, y of object, width, height of object
# [0.757391, 0.244731, 0.021406, 0.07822]
# rounding 6 decimal points
def convertBbox2YoloFormat(bbox, size):
  width, height = size
  x = round((bbox[0] + bbox[2]/2) / width, 6)
  y = round((bbox[1] + bbox[3]/2) / height, 6)
  w = round(bbox[2]/width, 6)
  h = round(bbox[3]/height, 6)
  return (x, y, w, h)

In [ ]:
train_txt = "train.txt"
train_txt_path = os.path.join(coco_path, train_txt)
valid_txt = "valid.txt"
valid_txt_path = os.path.join(coco_path, valid_txt)
trainfile = open(train_txt_path, 'w')
validfile = open(valid_txt_path, 'w')

In [ ]:
def BuildCustomDatasetFromCOCO(annFile, description_file):
    coco=COCO(annFile)
    # get Category Ids
    catNames = ['person', 'car', 'bus', 'truck']
    catIds = coco.getCatIds(catNms=catNames)
    # get Image Ids for 4 categories
    imgIds = []
    for catId in catIds:
      sub_imgIds = coco.getImgIds(catIds=catId)
      print(len(sub_imgIds))
      imgIds += sub_imgIds
    # get unique Image Ids
    imgIds = list(set(imgIds))
    
    # loop Image Ids
    # get Image Information form Coco dataset
    for imgId in imgIds:
        img_info = coco.loadImgs(ids = imgId)[0]
        # Load Image and annotation
        img = io.imread(img_info['coco_url'])
        annIds = coco.getAnnIds(imgIds=img_info['id'], catIds=catIds, iscrowd=0)
        anns = coco.loadAnns(annIds)

        # get file name, e.g. 000000262145.jpg
        filename = img_info['file_name']

        basename = os.path.splitext(filename)[0]
        txtfile_path = os.path.join(dataset_path, basename + '.txt')
        # write the image path in train.txt
        # example
        # coco/obj/000000262145.jpg
        # coco/obj/000000262146.jpg
        image_path = os.path.join(dataset_path, filename)
        description_file.write(image_path + "\n")

        # download image in coco/obj folder
        io.imsave(image_path, img)

        # write the yolo format bounding box in image.txt file
        size = (img_info['width'], img_info['height'])
        txtfile = open(txtfile_path, 'w')
        for i, ann in enumerate(anns):      
          bbox = convertBbox2YoloFormat(ann['bbox'], size)
          item_str = str(catIds.index(ann['category_id']))
          bbox_str = " ".join(str(entry) for entry in bbox)
          item_str += " " + bbox_str
          if i != len(anns) - 1:
            txtfile.write(item_str + "\n")
          else:
            txtfile.write(item_str)

In [ ]:
# initialize COCO api for instance annotations
dataDir='.'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)
BuildCustomDatasetFromCOCO(annFile, trainfile)
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
BuildCustomDatasetFromCOCO(annFile, validfile)

# Step 4: Write Custom Training Config for YOLOv4
cfg/yolov4-custom.cfg

cfg/coco.data

data/coco.names

In [ ]:
# Modify data/coco.names with our own categories
person
car
bus
truck

In [ ]:
# Modify cfg/coco.data
classes= 4
train  = data/coco/train.txt
valid  = data/coco/valid.txt
names = data/coco.names
backup = backup/

In [ ]:
# Modify cfg/yolov4-custom.cfg
# line 20, max_batches = 8000(4*2000)
# line 22, steps = 6400, 7200(0.8, 0.9*8000)
# change yolo layer classes to 4(class number), line 970, 1058, 1146
# change filters of convolution to 27((classes + 5)x3) immediately before each 3 yolo layers, line 963, 1051, 1139

# Step 5: Train the Model with Custom Dataset

In [ ]:
# train
./darknet detector train cfg/coco.data cfg/yolov4-custom.cfg yolov4.conv.137
# train with multiple GPU
# ./darknet detector train cfg/coco.data cfg/yolov4-custom.cfg yolov4.conv.137 -gpus 0,1,2,3
# If want to stop and restart training from a checkpoint:
# ./darknet detector train cfg/coco.data cfg/yolov4-custom.cfg backup/yolov3.backup -gpus 0,1,2,3

# Step 6:Infer Custom Objects with Saved YOLOv4 Weights

In [ ]:
#define utility function
def imShow(path):
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# test the images with trained yolov4 model
#test out our detector!
img_path = 'test.jpg'
!./darknet detect cfg/yolov4-custom.cfg backup/custom-yolov4-detector_last.weights {img_path} -dont-show
imShow('predictions.jpg')